In [200]:
from collections import OrderedDict

from PIL.ImageDraw import ImageDraw
from sqlalchemy import text

from app.database import StockChartEntity
from app.database.Core import SessionLocal, engine
import numpy as np
import pandas as pd
from PIL import Image,ImageDraw
from app.utils.Image import Scaler

In [239]:
 with engine.connect() as conn, conn.begin():
        stmt = text("""SELECT 
                A.*
                ,NVL(LAG(A.CLOSE) OVER(ORDER BY A.DATE,A.TIME),0) AS PREV_CLOSE
                ,AVG(A.CLOSE) OVER (ORDER BY A.DATE,A.TIME ROWS BETWEEN 113 PRECEDING AND CURRENT ROW) AS MIN_MA3
                ,B.DAILY_MA3
                FROM 
                TB_STOCK_CHART A
                JOIN (
                        SELECT
                            AVG(A.CLOSE) OVER (ORDER BY A.DATE,A.TIME ROWS BETWEEN 113 PRECEDING AND CURRENT ROW) AS DAILY_MA3
                            ,A.*
                        FROM (
                            SELECT
                                    A.DATE,
                                    A.CODE,
                                    A.TIME,
                                    MAX(HIGH) AS HIGH,
                                    MIN(LOW) AS LOW,
                                    SUM(VOLUME) AS VOLUME,
                                    (SELECT Z.CLOSE FROM TB_STOCK_CHART Z WHERE Z.DATE = A.DATE AND A.CODE = Z.CODE AND Z.TIME = MAX(A.TIME)) AS CLOSE,
                                    (SELECT Z.OPEN FROM TB_STOCK_CHART Z WHERE Z.DATE = A.DATE AND A.CODE = Z.CODE AND Z.TIME = MIN(A.TIME)) AS OPEN
                                FROM TB_STOCK_CHART A
                                WHERE
                                    A.CODE =:code
                                GROUP BY A.DATE,A.CODE
                             ) A
                ) B
                ON
                A.CODE = B.CODE AND A.DATE = B.DATE
                WHERE A.CODE = :code 
                ORDER BY DATE ASC , TIME ASC 
                LIMIT 0,1300
                """)
        params = {"code": "A000020"}
        df = pd.read_sql_query(stmt, con=conn, params=params)
        print(df)
        df["DATETIME"] = df["DATE"].astype("str") + df["TIME"].apply(lambda x: str(x).zfill(4))

        df_c = df.drop(["CODE", "DATE", "TIME"], axis=1)
        df_c["MA5"] = df_c['CLOSE'].rolling(window=5).mean()
        df_c.fillna(0, inplace=True)
        scaler = Scaler([800, 0], [df_c[["CLOSE", "OPEN", "LOW", "HIGH","DAILY_MA3","MIN_MA3"]].min().min(),
                                   df_c[["CLOSE", "OPEN", "LOW", "HIGH","DAILY_MA3","MIN_MA3"]].max().max()])
        df_sc = df_c[["CLOSE", "OPEN", "LOW", "HIGH","PREV_CLOSE","DAILY_MA3","MIN_MA3"]].apply(scaler,axis=1).astype("int")
        df_sc["DATETIME"] = df["DATETIME"]
        df_sc["width"] = 3900//df_sc.shape[0]
        df_sc["x"] = df_sc.index * df_sc["width"]
  
        # print(df_sc)

         CODE      DATE  TIME     OPEN     HIGH      LOW    CLOSE  \
0     A000020  20220411   901  13200.0  13200.0  13100.0  13150.0   
1     A000020  20220411   902  13100.0  13100.0  13100.0  13100.0   
2     A000020  20220411   903  13100.0  13200.0  13100.0  13100.0   
3     A000020  20220411   904  13200.0  13200.0  13150.0  13150.0   
4     A000020  20220411   905  13150.0  13150.0  13100.0  13100.0   
...       ...       ...   ...      ...      ...      ...      ...   
1295  A000020  20220415  1006  13100.0  13100.0  13100.0  13100.0   
1296  A000020  20220415  1007  13100.0  13100.0  13100.0  13100.0   
1297  A000020  20220415  1008  13100.0  13100.0  13100.0  13100.0   
1298  A000020  20220415  1010  13100.0  13150.0  13100.0  13150.0   
1299  A000020  20220415  1012  13150.0  13150.0  13150.0  13150.0   

      COMPARED_TO_THE_PREVIOUS_DAY  volume  TRADING_VALUE  ...  \
0                              0.0  5055.0     66490000.0  ...   
1                              0.0     

In [240]:
x1 = 2
y1 = 4
x2 = 4
y2 = 10

def getY(xl,yl,x):
        return (yl[1]-yl[0])/(xl[1] - xl[0]) * (x - xl[0]) + yl[0]






In [241]:
img = Image.new("RGB", (3900, 800), (0, 0, 0))
draw = ImageDraw.Draw(img)
m2 =0
cl = 0

ma3 = []
minma3 = []
for i in range(df_sc.shape[0]):
        y = df_sc.loc[i,"CLOSE"]
        h = df_sc.loc[i,"OPEN"]
        w = df_sc.loc[i,"width"]
        
        
        x = i*w
        x2 = x+w
        x3 = x+(x2-x)//2 -2
        x4 = x3+4
        
        pc = df_sc.loc[i,"PREV_CLOSE"]
        lp = df_sc.loc[i,"LOW"]
        hp = df_sc.loc[i,"HIGH"]
        z1 = df_sc.loc[i,"DAILY_MA3"]
        mz1 = df_sc.loc[i,"MIN_MA3"]
        ma3 += [x3,z1]
        minma3 += [x3,mz1]
        if y-h >m2:
                m2 = y-h
                cl = df_sc.loc[i,"DATETIME"]
        if y>h:
                color = (0,0,255)
                y,h = h,y
        else:
                color = (255,0,0)
        if y==h:
               if y<pc:
                       color = (255,0,0)
               else:
                       color = (0,0,255)
     
        if y==h:
                h = y+1
        
       
        try:
                draw.rectangle([(x,y),(x2,h)],color)
                draw.rectangle([(x3,hp),(x4,lp)],color)
        except Exception as e:
                print([(x3,x4),(y,h)],str(e))
        
draw.line(ma3,(0,255,0))
draw.line(minma3,(100,255,100))
                

In [242]:
img.show()